<a href="https://colab.research.google.com/github/reema-dass26/Electricity_utilization_data_management/blob/main/mqtt_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openpyxl


In [ ]:
import pandas as pd
import uuid
from datetime import datetime
import argparse
# Setup argparse to accept command line arguments
file_path = input("Please enter the path to your Excel file: ")

# Load the Excel file without headers
try:
    df = pd.read_excel(file_path, header=None)
    print("Excel file loaded successfully.")
except Exception as e:
    print(f"Error loading the Excel file: {e}")
    exit()

interval_of_logging=15
# Step 1: Find the index of the row where the 'Spaltensumme' starts
start_row_index = df[df[0] == 'Spaltensumme / minimale Qualität'].index[0]

# Step 2: Extract rows starting from the next row after the marker, and reset index
df_cleaned = df.iloc[start_row_index + 1:].reset_index(drop=True)

# Step 3: Select only the first two columns (timestamps and corresponding values)
df_cleaned = df_cleaned[[0, 1]]

# Step 4: Rename the columns as per voltaware.
df_cleaned.columns = ['timestampUtcEpochSecs', 'consumptionKwh']
print(type(df_cleaned ['timestampUtcEpochSecs']))
#validating the total consumption( for internal testing)
total_sum = df_cleaned['consumptionKwh'].sum()
print(f"Sum of the 'consumptionKwh' column: {total_sum}")

#uuid generation for device id
myuuid = uuid.uuid4()
print('Your UUID is: ' + str(myuuid))

# Cleaned values (convert timestampUtcEpochSecs to a list of integers) and convert the structure to list of objects
cleaned_values = df_cleaned[['timestampUtcEpochSecs', 'consumptionKwh']].apply(lambda row: {
    'timestampUtcEpochSecs': int(row['timestampUtcEpochSecs'].timestamp()),  # Convert Unix timestamp to integer
    'consumptionKwh': row['consumptionKwh']
}, axis=1).tolist()


new_data = {
  	'deviceId' : str(myuuid),
    'dataFrequencyMins' :interval_of_logging,
    'date' :  str(datetime.now().strftime('%Y-%m-%d')),
    "electricReadings": cleaned_values
}

# Define the output file path
output_file_path = str(myuuid)

with open(output_file_path, 'w') as f:
    json.dump(new_data, f, indent=4)

print(f"New JSON file created: {output_file_path}")



Please enter the path to your Excel file: /content/sample_data/Report_20241211_20241214_202501101235.xlsx
Excel file loaded successfully.
<class 'pandas.core.series.Series'>
Sum of the 'consumptionKwh' column: 0.08200000000000003
Your UUID is: e81874d3-04a5-4c6b-9a86-0410accb4af6
New JSON file created: e81874d3-04a5-4c6b-9a86-0410accb4af6


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# New section